In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

from hydra import compose, initialize
import matplotlib.pyplot as plt
import os
from tqdm.auto import tqdm
import h5py
import numpy as np
from UniversalUnconsciousness.data_utils import *
from UniversalUnconsciousness.plot_utils import *
from UniversalUnconsciousness.power_analysis import *

plt.style.use('UniversalUnconsciousness.sci_style')
load_font()

In [3]:
with initialize(version_base="1.3", config_path="../UniversalUnconsciousness/DeLASE_analysis/conf"):
    cfg = compose(config_name="config")

In [4]:
cfg.params.low_pass = None
cfg.params.high_pass = None

In [5]:
anesthetic_agent_list = cfg.plotting.anesthetic_agent_list
img_save_dir = cfg.plotting.img_save_dir
curve_colors = cfg.plotting.curve_colors
loc_roc_colors = cfg.plotting.loc_roc_colors
os.makedirs(img_save_dir, exist_ok=True)

In [6]:
anesthetic_agent_list = anesthetic_agent_list[1:]

In [7]:
verbose = False
agent_data = {}
for data_class, agent in tqdm(anesthetic_agent_list):
    if agent == 'propofol':
        cfg.params.grid_set = 'grid_set5'
    else:
        cfg.params.grid_set = 'grid_set4'
    cfg.params.data_class = data_class
    if 'propofol' in cfg.params.data_class:
        session_list = [f[:-4] for f in os.listdir(os.path.join(cfg.params.all_data_dir, 'anesthesia', 'mat', cfg.params.data_class)) if f.endswith('.mat')]
    else:
        session_list = [f[:-4] for f in os.listdir(os.path.join(cfg.params.all_data_dir, cfg.params.data_class, 'mat')) if f.endswith('.mat')]
        session_list = [session for session in session_list if session not in ['PEDRI_Ketamine_20220203']]
        session_list = [session for session in session_list if agent.lower()[:3] in session.lower()]
    
    areas = ['all']
    agent_data[(data_class, agent)] = {'session_list': session_list}

    session_lists, locs, rocs, ropaps = get_session_plot_info(cfg, session_list, verbose=False)

    agent_data[(data_class, agent)]['session_lists'] = session_lists
    agent_data[(data_class, agent)]['locs'] = locs
    agent_data[(data_class, agent)]['rocs'] = rocs
    agent_data[(data_class, agent)]['ropaps'] = ropaps

    noise_filter_info = get_noise_filter_info(cfg, session_list, verbose=verbose)
    agent_data[(data_class, agent)]['noise_filter_info'] = noise_filter_info
    
    pca_chosen = get_pca_chosen(cfg, session_list, areas, noise_filter_info, verbose=verbose)
    agent_data[(data_class, agent)]['pca_chosen'] = pca_chosen
    all_indices_to_run = collect_grid_indices_to_run(cfg, session_list, areas, noise_filter_info, pca_chosen, verbose=verbose)
    if all_indices_to_run:
        raise ValueError(f"Sessions for agent {agent} have incomplete grid search - cannot continue")
    
    grid_params_to_use = get_grid_params_to_use(cfg, session_list, areas, noise_filter_info, pca_chosen, verbose=verbose)
    agent_data[(data_class, agent)]['grid_params_to_use'] = grid_params_to_use
    all_indices_to_run = collect_delase_indices_to_run(cfg, session_list, areas, noise_filter_info, pca_chosen, grid_params_to_use, verbose=verbose)
    
    if all_indices_to_run:
        raise ValueError(f"Sessions for agent {agent} have incomplete DeLASE - cannot continue")

    delase_results = get_delase_results(cfg, session_list, areas, grid_params_to_use, pca_chosen, verbose=verbose)
    agent_data[(data_class, agent)]['delase_results'] = delase_results

  0%|          | 0/2 [00:00<?, ?it/s]

Only 0 valid windows could be found for section 'awake lever1' with times [np.float64(-36.23385444444445), np.float64(-26.050830555555557)]
Only 3 valid windows could be found for section 'early unconscious' with times [np.float64(26.003301111111107), 45]
Only 1 valid windows could be found for section 'awake lever1' with times [np.float64(-36.120676111111116), np.float64(-26.045618333333337)]
Only 3 valid windows could be found for section 'awake lever1' with times [np.float64(-36.10079944444445), np.float64(-26.03974944444445)]
Only 0 valid windows could be found for section 'awake lever2' with times [np.float64(-10.029036111111115), 0]


In [8]:
area = 'all'
# save_dir = "/orcd/data/ekmiller/001/eisenaj/UniversalUnconsciousness/power_analysis"
save_dir = "/scratch2/weka/millerlab/eisenaj/UniversalUnconsciousness/power_analysis"

In [9]:
top_percent = 0.1
for data_class, agent in tqdm(anesthetic_agent_list):
    cfg.params.data_class = data_class
    delase_results = agent_data[(data_class, agent)]['delase_results']
    agent_save_dir = os.path.join(save_dir, data_class, agent, area)
    os.makedirs(agent_save_dir, exist_ok=True)
    for session in tqdm(agent_data[(data_class, agent)]['session_list']):
        if not os.path.exists(os.path.join(agent_save_dir, f'{session}__top_percent_{top_percent}_power_analysis.pkl')):
            print(f"Performing power analysis for {session}...")
            freq_powers, freq_r2_scores = perform_power_analysis(delase_results, cfg, session, area, top_percent=top_percent, verbose=True)
            ret_dict = {'freq_powers': freq_powers, 'freq_r2_scores': freq_r2_scores}
            pd.to_pickle(ret_dict, os.path.join(agent_save_dir, f'{session}__top_percent_{top_percent}_power_analysis.pkl'))
        else:
            print(f"Power analysis for {session} already exists")



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Power analysis for SPOCK_Ketamine_20210907 already exists
Power analysis for PEDRI_Ketamine_20220106 already exists
Power analysis for SPOCK_Ketamine_20210730 already exists
Power analysis for PEDRI_Ketamine_20220111 already exists
Power analysis for SPOCK_Ketamine_20210727 already exists
Power analysis for SPOCK_Ketamine_20210712 already exists
Power analysis for PEDRI_Ketamine_20220222 already exists
Power analysis for SPOCK_Ketamine_20210707 already exists
Power analysis for SPOCK_Ketamine_20210831 already exists
Power analysis for PEDRI_Ketamine_20220215 already exists
Power analysis for PEDRI_Ketamine_20220217 already exists
Power analysis for SPOCK_Ketamine_20210827 already exists
Power analysis for PEDRI_Ketamine_20220211 already exists
Power analysis for PEDRI_Ketamine_20220224 already exists
Power analysis for PEDRI_Ketamine_20220128 already exists
Power analysis for SPOCK_Ketamine_20210719 already exists
Power analysis for PEDRI_Ketamine_20220201 already exists
Power analysis

  0%|          | 0/32 [00:00<?, ?it/s]

Power analysis for SPOCK_Dexmedetomidine_20211028 already exists
Power analysis for PEDRI_Dexmedetomidine_20220419 already exists
Power analysis for PEDRI_Dexmedetomidine_20220329 already exists
Power analysis for PEDRI_Dexmedetomidine_20220324 already exists
Power analysis for SPOCK_Dexmedetomidine_20210923 already exists
Power analysis for SPOCK_Dexmedetomidine_20211123 already exists
Power analysis for SPOCK_Dexmedetomidine_20211020 already exists
Power analysis for SPOCK_Dexmedetomidine_20211012 already exists
Power analysis for PEDRI_Dexmedetomidine_20220308 already exists
Power analysis for PEDRI_Dexmedetomidine_20220315 already exists
Power analysis for PEDRI_Dexmedetomidine_20220405 already exists
Power analysis for SPOCK_Dexmedetomidine_20211111 already exists
Power analysis for PEDRI_Dexmedetomidine_20220310 already exists
Power analysis for SPOCK_Dexmedetomidine_20210916 already exists
Power analysis for PEDRI_Dexmedetomidine_20220427 already exists
Power analysis for PEDRI_